# Example uses of Mirage catalog generators

In [ ]:
import os
import numpy as np
from mirage.catalogs import catalog_generator
from mirage.catalogs import create_catalog

#### Table of Contents

* [Create Catalogs with user-input data](#catalog_classes)
   * [Point Source](#point_source)
   * [Galaxy](#galaxy)
   * [Extended Sources](#extended)
   * [Moving Point Source](#moving_ptsrc)
   * [Moving Sersic](#moving_sersic)
   * [Moving Extended](#moving_extended)
   * [Non-sidereal](#non_sidereal)
* [Create Catalogs via Database Queries](#queries)
   * [Besancon model query](#besancon_query)
   * [Point Source Catalogs from 2MASS, Gaia, WISE, and Besancon](#get_all_catalogs)
      * [Convenience Functions for General Pointings](#convenience)
   * [Representative Extragalactic Catalogs](#galaxy_background)
* [Create Catalogs from an input APT File](#from_apt)
* [Additional Functionality](#additional_functionality)
   * [Combining Catalogs Using the `add_catalog` Method](#combining_add_catalog)
   * [Combining Catalogs Using the `combine_catalogs` Function](#combining_combine)

Define paths to help organize input data

In [ ]:
path = os.path.dirname(catalog_generator.__file__)
input_data_path = os.path.abspath(os.path.join(path, '../../examples/'))

<a id='catalog_classes'></a>
## Create catalogs with user-input data 

<a id='point_source'></a>
### PointSource

Catalogs containing point source objects can be created using the PointSourceCatalog class. The user must provide lists or numpy arrays containing values for Right Ascention, Declination (or detector pixel x and y), along with magnitudes in a particular filter. 

In [ ]:
ra = np.random.random(5) + 80.
dec = np.random.random(5) - 69.8
mags1 = np.random.random(5) + 17.
mags2 = np.random.random(5) + 18.5

In [ ]:
ptsrc = catalog_generator.PointSourceCatalog(ra=ra, dec=dec)

RA and Dec are attributes of the class.

In [ ]:
print('RA: {}'.format(ptsrc.ra))
print('Dec: {}'.format(ptsrc.dec))

Magnitude information is added via the `add_magnitude_column` method. The instrument and filter associated with the magnitude list must be supplied as well. These two pieces of information are combined to create the magnitude column name in the resulting table. Note that multiple instruments and filters can be present in a single catalog.

The magnitude columns in a given catalog must all be in the same magnitude system. Allowed values for the magnitude system include `abmag`, `stmag`, and `vegamag`. If no system is given, the default value of `abmag` is used.

In [ ]:
ptsrc.add_magnitude_column(mags1, instrument='nircam', filter_name='f090w', magnitude_system='abmag')
ptsrc.add_magnitude_column(mags1+0.1, instrument='nircam', filter_name='f444w', magnitude_system='abmag')
ptsrc.add_magnitude_column(mags2, instrument='niriss', filter_name='f200w', magnitude_system='abmag')

To view the table, use the `table` attribute.

In [ ]:
ptsrc.table

Save the table to an ascii file. When a catalog is saved, information on the location units and magnitude system is saved to the file such that Mirage can make use of it.

In [ ]:
ptsrc.save('ptsrc_test.cat')

The units of the input positions are determined by whether they are entered via the `ra` and `dec` keywords, or the `x` and `y` keywords. This information is used by Mirage when creating the simulated data.

In [ ]:
ptsrc.location_units

Here is an example using pixel (x,y) coordinates as input rather than RA, Dec

In [ ]:
x_pix = np.random.random(5) * 2048
y_pix = np.random.random(5) * 2048
ptsrc_pix = catalog_generator.PointSourceCatalog(x=x_pix, y=y_pix)
ptsrc_pix.add_magnitude_column(mags1, instrument='nircam', filter_name='f090w')

In [ ]:
ptsrc_pix.location_units

In [ ]:
ptsrc_pix.table

<a id='galaxy'></a>
### Galaxy

The GalaxyCatalog class creates catalogs of extragalactic sources. These catalogs are similar to the PointSourceCatalog, with the added columns of ellipticity, radius, sersic_index, and position_angle. `radius` input values can be in units of arcseconds or pixels. The units are specified using the `radius_units` keyword, which can be set to `arcsec` or `pixels`.

In [ ]:
radius = np.random.random(5) + 0.5
ellip = np.random.random(5) + 0.45
posang = np.random.random(5) + 27.
sersic = np.random.random(5) + 3.3

In [ ]:
gal = catalog_generator.GalaxyCatalog(ra=ra, dec=dec, ellipticity=ellip, radius=radius, sersic_index=sersic,
                                      position_angle=posang, radius_units='arcsec')

In [ ]:
gal.add_magnitude_column(mags1, instrument='nircam', filter_name='f444w', magnitude_system='stmag')

In [ ]:
gal.table

In [ ]:
gal.save('galaxy_test.cat')

<a id='extended'></a>
### Extended Sources

Catalogs containing extended sources, which are essentially fits files containing stamp images to be added to the simulated data, are created using the ExtendedCatalog class. Again, this is functionally similar to the PointSourceCatalog class, with added columns for filenames and position angles.

In [ ]:
filenames = ['f{}.fits'.format(str(i)) for i in range(5)]
pos_angles = np.random.random(5) + 14.5

In [ ]:
extend = catalog_generator.ExtendedCatalog(filenames=filenames, ra=ra, dec=dec, position_angle=pos_angles)

Note that for extended sources, it is possible not to specify a magnitude value. If a magnitude is supplied, the contents of the fits file will be read in and scaled based on that magnitude. If no magnitude is given (which can be done by having any non-number value in the catalog), then the contents of the fits file are assumed to be in counts per second. The image is then directly added to the simulated data as-is. Or, if no magnitude is given, the data in the fits file can also be scaled by a multiplicative factor. This is contolled from the `extendedScale` entry in the input yaml file for the observation, rather than within the catalog. Note that in this case, the same scaling factor will be applied to all external sources in the catalog that have no magnitude given.

In [ ]:
mags3 = list(np.random.random(5) + 13.2)
mags3[2] = 'none'

In [ ]:
extend.add_magnitude_column(mags3, instrument='nircam', filter_name='f480m', magnitude_system='abmag')

In [ ]:
extend.table

In [ ]:
extend.save('extended_test.cat')

<a id='moving_ptsrc'></a>
### Moving Point Sources (e.g. KBOs)

Moving sources, such as asteroids/KBOs can also be added to simulated data, using the MovingPointSourceCatalog class. Inputs to this class are similar to those for the PointSouceCatalog class, with additional entries for ra_velocity and dec_velocity, or x_velocity and y_velocity. Velocities can be given in untits of arcseconds per hour or pixels per hour. The units for a given catalog instance are determined by whether the veolcities are provided via the ra_velocity, dec_velocity keywords or the x_velocity, y_velocity keywords.

In [ ]:
ra_vel = np.random.random(5)*0.1 + 0.432
dec_vel = np.random.random(5)*0.15 + 0.875

In [ ]:
mv_ptsrc = catalog_generator.MovingPointSourceCatalog(ra=ra, dec=dec, ra_velocity=ra_vel, dec_velocity=dec_vel)

In [ ]:
mv_ptsrc.add_magnitude_column(mags1, instrument='nircam', filter_name='f444w')
mv_ptsrc.add_magnitude_column(mags2, instrument='niriss', filter_name='f200w')

In [ ]:
mv_ptsrc.velocity_units

In [ ]:
mv_ptsrc.table

In [ ]:
mv_ptsrc.save('moving_point_source_test.cat')

<a id='moving_sersic'></a>
### Moving Sersic Sources

Moving Sersic sources can be added to a simulation using the MovingSersicCatalog class. This may be useful for placing a resolved planet/moon/asteroid that JWST is not tracking within the field of view. This class has all of the attributes of the GalaxyCatalog class, as well as those from the MovingPointSourceCatalog class.

In [ ]:
mv_sersic = catalog_generator.MovingSersicCatalog(ra=ra, dec=dec, ra_velocity=ra_vel, dec_velocity=dec_vel,
                                                  ellipticity=ellip, radius=radius, sersic_index=sersic,
                                                  position_angle=posang, radius_units='arcsec')                                                  

In [ ]:
mv_sersic.add_magnitude_column(mags2, instrument='nircam', filter_name='f090w')

In [ ]:
mv_sersic.table

In [ ]:
mv_sersic.save('moving_sersic_source_test.cat')

<a id='moving_extended'></a>
### Moving Extended Sources

Extended sources that move through the field of view over time can also be added to simulations, using the MovingExtendedCatalog class. Inputs here are the same as those in the ExtendedCatalog and MovingPointSourceCatalog classes combined.

In [ ]:
mv_ext = catalog_generator.MovingExtendedCatalog(ra=ra, dec=dec, ra_velocity=ra_vel, dec_velocity=dec_vel,
                                                 filenames=filenames, position_angle=pos_angles)

In [ ]:
mv_ext.add_magnitude_column(mags3, instrument='nircam', filter_name='f090w')

In [ ]:
mv_ext.table

In [ ]:
mv_ext.save('moving_extended_source_test.cat')

<a id='non_sidereal'></a>
### Non-Sidereal Sources

The final type of catalog that is accepted by Mirage is that for non-sidereal sources. A source placed in this catalog will cause Mirage to produce data as if JWST was tracking on that source during the observation. (In this case, if you want to have background objects (stars, galaxies, etc) trailing through the field of view, use MovingPointSourceCatalog, MovingSersicCatalog, and MovingExtendedCatalogs to define those.)

The NonSiderealCatalog takes as input the position RA, Dec (or detector x, y), the velocity in the RA, and Dec directions (in arcsec per hour or pixels per hour), as well as a column defining what type of object it is. Allowed values for the object type include: 'pointSource' for point sources, 'sersic' for 2D Sersic profile objects, and any other value for an extended object.

In [ ]:
initial_ra = [80.5]
initial_dec = [-69.7]
ra_vel = [0.4]
dec_vel = [0.03]
ob_type_list = ['pointSource']
ns = catalog_generator.NonSiderealCatalog(ra=initial_ra, dec=initial_dec, ra_velocity=ra_vel, dec_velocity=dec_vel,
                                          object_type=ob_type_list)

In [ ]:
ns_mag = [13.5]
ns.add_magnitude_column(ns_mag, instrument='nircam', filter_name='f480m', magnitude_system='abmag')

In [ ]:
ns.table

In [ ]:
ns.save('nonsidereal_source_test.cat')

<a id='queries'></a>
## Create Catalogs via Database Queries

Mirage also contains functionality to generate source catalogs through queries to certain astronomical databases. These include 2MASS, GAIA, and WISE, as well as queries to the Besancon galaxy model. These queries are all accomplished using the [`astroquery`](https://astroquery.readthedocs.io/en/latest/) package, with the exception of the Besancon query.

<a id='besancon_query'></a>
### Besancon model query

Below we show how to submit a query to the Besancon model. This is completed using a wrapper around a client program authored by the [group who maintain the Besancon model](https://model.obs-besancon.fr/modele_home.php). First, you must [create a user account](https://model.obs-besancon.fr/modele_home.php). Once your account is approved, you can query the model.

The required inputs are: 
  - RA and Dec of the pointing (in degrees)
  - Width of the box (in arcseconds) on the sky to query, 
  - User name for your account on the [Besancon model page](https://model.obs-besancon.fr/modele_home.php)
  - Optionally, the K magnitude limits for stars to retrieve (default is 13 - 29)

In [ ]:
ra = 80.4  # degrees
dec = -69.8  # degrees
box_width = 120  # arcseconds

In [ ]:
create_catalog.besancon(ra, dec, box_width, username='hilbert', kmag_limits=(17, 30))

After the query is submitted, you will receive an email containing a link to download the resulting catalog file. For the purposes of this example, we will use the file below.

In [ ]:
besancon_result = os.path.join(input_data_path, 'besancon_query_result.cat')

NOTE: The code is general enough that if you can replace `besancon_result` with any ascii source catalog, as long as it contains columns for RA and Dec, along with a `K` column containing K band magnitudes, and an `Av` column containing ISM extinction values. In addition, it must contain columns corresponding to one of these two options:

  1. `J`, `H`,  and `V` columns with appropriate magnitudes for those Johnson-Cousins bands, or
  2. `V-K`, `J-K`, and `J-H` columns with color values.

<a id='get_all_catalogs'></a>
### Point Source Catalogs from 2MASS, Gaia, WISE, and Besancon

The first of the two main catalog-generating functions is `get_all_catalogs`. This will query the [2MASS 'fp_psc' catalog](https://astroquery.readthedocs.io/en/latest/irsa/irsa.html), [GAIA](https://astroquery.readthedocs.io/en/latest/gaia/gaia.html), and [WISE 'allsky_4band_p3as_psd'](https://astroquery.readthedocs.io/en/latest/irsa/irsa.html) databases, and cross-reference the results. In addition, the Besancon model will be queried, and a representative population of background stars will be returned. The magnitudes for all returned sources will be converted into the those for the requested JWST filters.

Input parameters for the `get_all_catalogs` function include the central RA and Dec (in decimal degrees) for the catalog, as well as the width, in arcseconds, of the field of view to consider. A JWST instrument name and accompanying filter list is also required. Finally, in order to query the Besancon model, a valid email address is necessary.

In [ ]:
filter_list = ['F444W', 'F480M']
cat, mag_column_names = create_catalog.get_all_catalogs(ra, dec, box_width,
                                                        besancon_catalog_file=besancon_result,
                                                        instrument='NIRCAM', filters=filter_list,
                                                        ra_column_name='RAJ2000', dec_column_name='DECJ2000')

Note that in this case, the VJHKL source magnitudes returned by the Besancon model are included in the output catalog in addition to those in the requested JWST filters.

In [ ]:
cat.table

In [ ]:
# Save catalog so it can serve as input to Mirage later
cat.save('GAIA_2MASS_WISE_BESANCON_sources.cat')

<a id='convenience'></a>
### Convenience Functions for General Pointings

There are also several convenience functions to get representative catalogs looking into the galactic plane, out of the galactic plane, and into the galactic bulge. Note that in these examples, we continue to use `besancon_result`. However, since that catalog was generated around a particular pointing, it will not actually contain sources at the RA and Dec of e.g. the galactic plane. 

To properly create the catalogs below, you will have to query the Besancon model with the appropriate pointing for each. In the convenience functions below, the RA and Dec values used are:

  - Galactic plane:        Galactic Longitude: 45.0 Galactic Latitude: 0.0,  RA: 288.42, Dec: 10.72 
  - Out of Galactic Plane: Galactic Longitude: 45.0 Galactic Latitude: 85.0, RA: 198.40, Dec: 28.06
  - Galactic Bulge:        Galactic Longitude: 0.0  Galactic Latitude: 5.0,  RA: 261.65, Dec: -26.25

In [ ]:
filter_list = ['F444W', 'F480M']
instrument = 'NIRCAM'
galactic_plane = create_catalog.galactic_plane(box_width, instrument, filter_list,
                                               besancon_result, ra_column_name='RAJ2000',
                                               dec_column_name='DECJ2000')

In [ ]:
galactic_plane.table

In [ ]:
galactic_plane.save('galactic_plane.cat')

In [ ]:
filter_list = ['F444W', 'F480M']
instrument = 'NIRCAM'
galactic_bulge = create_catalog.galactic_bulge(box_width, instrument, filter_list,
                                               besancon_result, ra_column_name='RAJ2000',
                                               dec_column_name='DECJ2000')

In [ ]:
galactic_bulge.table

In [ ]:
galactic_bulge.save('galactic_bulge.cat')

In [ ]:
filter_list = ['F444W', 'F480M']
instrument = 'NIRCAM'
out_of_plane = create_catalog.out_of_galactic_plane(box_width, instrument, filter_list,
                                                    besancon_result, ra_column_name='RAJ2000',
                                                    dec_column_name='DECJ2000')

In [ ]:
out_of_plane.table

In [ ]:
out_of_plane.save('out_of_plane.cat')

<a id='galaxy_background'></a>
### Representative Extragalactic Catalogs

A catalog of extra-galactic sources can also be generated using the `galaxy_background` function. This will extract the sources from the GOODS-S 3DHST catalog in an area matching the size of your requested area. The magnitudes of these sources will then be converted to magnitudes in the input JWST filters. The requested area can be a box of width `box_width` or a circle of radius `box_width`, depending on the value of `boxflag` provided (True=box, False=circle)

In [ ]:
center_ra = 80.1
center_dec = -69.7
v3_angle = 0.  # degrees
width = 100  # arcseconds
instrument = 'nircam'
filter_list = ['F444W', 'F480M']
background_galaxy_catalog, used_seed_value = create_catalog.galaxy_background(center_ra, center_dec, v3_angle,
                                                                              width, instrument, filter_list,
                                                                              boxflag=False, brightlimit=14.0)

In [ ]:
background_galaxy_catalog.table

In [ ]:
background_galaxy_catalog.save('background_galaxies_from_3DHST.cat')

<a id='from_apt'></a>
## Create Catalogs from an input APT File

Finally, Mirage contains the `create_catalog.for_proposal` function that can be used to create point source and galaxy catalogs from an [APT](https://jwst-docs.stsci.edu/display/JPP/JWST+Astronomers+Proposal+Tool+Overview) file. This function collects the target RA and Dec values from the proposal, as well as the list of instruments and filters used for the observations. It then runs `get_all_catalogs` and `galaxy_background` to produce point source and galaxy catalogs. These catalogs can then be used as input when producing the yaml files needed to run Mirage.

The inputs needed from the APT file include the xml and pointing files. These can both be exported and saved when running APT.

In [ ]:
xml_file = os.path.join(input_data_path, 'apt_data', 'apt_1071.xml')
pointing_file = xml_file.replace('.xml', '.pointing')
output_dir = './'
ptsrc_cat, gal_cat, ptsrc_names, \
   gal_names, pmap, gmap = create_catalog.for_proposal(xml_file, pointing_file,
                                                       point_source=True,
                                                       extragalactic=True,
                                                       catalog_splitting_threshold=0.12,
                                                       besancon_catalog_file=besancon_result,
                                                       out_dir=output_dir,
                                                       save_catalogs=True)

The objects returned by the `for_proposal` function include a list of PointSourceCatalog objects, a list of GalaxyCatalog objects, a list of file names to which the each type of catalog was saved. The lists of filenames are empty if the user chooses not to save the catalogs to files.

In [ ]:
ptsrc_cat

In [ ]:
ptsrc_names

In [ ]:
gal_cat

In [ ]:
gal_names

In [ ]:
ptsrc_cat[0].table

Also returned is a list of which catalog names are associated with which observation numbers in the proposal

In [ ]:
# Keys are the observation numbers. Values are the ptsrc catalog files.
pmap

In [ ]:
# Keys are the observation numbers. Values are the galaxy catalog files.
gmap

<a id='additional_functionality'></a>
## Additional Functionality

<a id='combining_add_catalog'></a>
### Combining Catalogs Using the `add_catalog` Method

There are two ways to combine two existing catalogs. The first is through the `add_catalog` method in the various catalog object classes. This method simply adds the sources from one catalog to the bottom of the list of sources in another catalog. It then copies the magnitude columns from the first catalog into the second, and uses a fill value (currently 99) to populate magnitude entries where there is no information. Note that the position units (RA, Dec or x, y) and the magnitude system of the two catalogs must match in order for the catalogs to be combined.

In [ ]:
ra1 = np.random.random(3) + 80.
dec1 = np.random.random(3) -69.7
mags1 = np.random.random(3) + 15.
mags1a = np.random.random(3) + 17.
ptsrc1 = catalog_generator.PointSourceCatalog(ra=ra1, dec=dec1)
ptsrc1.add_magnitude_column(mags1, instrument='nircam', filter_name='f444w')
ptsrc1.add_magnitude_column(mags1a, instrument='nircam', filter_name='f480m')

In [ ]:
ra2 = np.random.random(3) + 80.1
dec2 = np.random.random(3) -69.6
mags2 = np.random.random(3) + 15.
mags2a = np.random.random(3) + 19.
ptsrc2 = catalog_generator.PointSourceCatalog(ra=ra2, dec=dec2)
ptsrc2.add_magnitude_column(mags2, instrument='nircam', filter_name='f444w')
ptsrc2.add_magnitude_column(mags2a, instrument='nircam', filter_name='f470n')

In [ ]:
ptsrc1.table

In [ ]:
ptsrc2.table

In [ ]:
# Default magnitude fill value is 99
ptsrc1.add_catalog(ptsrc2, magnitude_fill_value=99.)

In [ ]:
ptsrc1.table

<a id='combining_combine'></a>
### Combining Catalogs Using the `combine_catalogs` Function

The other way to combine catalogs is using the `create_catalog.combine_catalogs` function. This function creates a new catalog that contains the combined contents of the two input catalogs. In this case, 

In [ ]:
ra1 = np.random.random(3) + 80.
dec1 = np.random.random(3) -69.7
mags1 = np.random.random(3) + 15.
mags1a = np.random.random(3) + 17.
ptsrc1 = catalog_generator.PointSourceCatalog(ra=ra1, dec=dec1)
ptsrc1.add_magnitude_column(mags1, instrument='nircam', filter_name='f444w')
ptsrc1.add_magnitude_column(mags1a, instrument='nircam', filter_name='f480m')

In [ ]:
ra2 = np.random.random(3) + 80.1
dec2 = np.random.random(3) -69.6
mags2 = np.random.random(3) + 15.
mags2a = np.random.random(3) + 19.
ptsrc2 = catalog_generator.PointSourceCatalog(ra=ra2, dec=dec2)
ptsrc2.add_magnitude_column(mags2, instrument='nircam', filter_name='f444w')
ptsrc2.add_magnitude_column(mags2a, instrument='nircam', filter_name='f470n')

In [ ]:
# As with the add_catalog method, the default magnitude fill value is 99
ptsrc3 = create_catalog.combine_catalogs(ptsrc1, ptsrc2, magnitude_fill_value=99.)

In [ ]:
ptsrc3.table